### Generate stimulus:

In [175]:
import numpy as np
from scipy.signal import convolve2d

actBits = 3
weightBits = 3

nActs = 16
nWeights = 3
nOuts = nActs - nWeights + 1

a = np.random.randint(-2**(actBits-1),2**(actBits-1)-1,size=(nActs,nActs))
w = np.random.randint(-2**(weightBits-1),2**(weightBits-1)-1,size=(nWeights,nWeights))
o = convolve2d(a,w[::-1].T[::-1].T,mode='valid')

print(f'PE cluster must be {nOuts} PEs wide and {nWeights} PEs high ')

PE cluster must be 14 PEs wide and 3 PEs high 


In [176]:
a

array([[-1, -2, -1,  2, -2,  0,  0,  1, -4, -1, -2, -3,  1,  1,  1,  1],
       [-3, -1, -2, -1, -4,  1, -4,  0, -4, -3, -3, -1, -1, -4, -2, -2],
       [-1, -1, -3, -4,  0, -4,  2, -4,  1,  0, -3,  2, -2, -4,  1, -3],
       [-2, -4,  1,  1, -1,  1, -2,  0,  1,  2, -3, -4, -3,  1, -1,  1],
       [ 1,  0, -4,  1, -3,  1, -4, -3, -2,  1, -4,  2,  1,  0,  1, -1],
       [-3,  1,  1,  1, -3,  1,  2, -2, -3,  2, -2,  1,  0, -2, -3, -2],
       [ 0,  1,  2,  0,  2, -1,  0, -2, -4, -3,  0,  1,  1,  1, -1,  2],
       [ 1,  1,  2, -1,  0,  1,  1, -3,  0, -2, -3, -3,  2,  0,  2, -4],
       [-2, -4, -3, -4,  2, -3, -4,  1,  2,  2,  2, -4,  2,  1,  2, -4],
       [-3, -2,  0, -3,  2,  0,  0,  1, -4, -3,  2, -3,  1,  1, -2, -1],
       [-4,  0, -4, -3,  1,  0, -4,  1,  2, -2, -3,  0, -3,  0,  2,  1],
       [-4, -4,  1,  2,  0,  0,  2,  2, -3, -3,  0,  2, -3,  2, -3,  2],
       [-4,  1,  0, -1, -3,  2, -4, -2, -3, -1,  0,  1, -1,  2, -3,  0],
       [ 0, -4,  0, -2, -4, -3, -4, -1, -2,  1,  1,

In [177]:
w

array([[-4,  1, -4],
       [ 0, -2, -4],
       [-4, -2, -1]])

In [178]:
o

array([[ 25,  21,  52,  14,  28,  20,  32,  30,  40,  28,  17,  27,  24,
         22],
       [ 48,  41,  26,   9,  37,   4,  43,   2,  32,  13,  42,  60,  23,
         29],
       [ 19,  18,  27,  31,   8,  45,   4,  22,  26,  13,  53,  -2,  -3,
         26],
       [ 25,  10,   8,  -6,  35,   8,  17,   5,  34,   0,  18,   5,  20,
          9],
       [  2, -22,  29, -12,  13,  14,  45,  23,  51,  -5,   9,  -6,   3,
         15],
       [ -9, -18,  -5,  -8,   4,  11,  24,  31,  35,  -1,  19,   6,   2,
          7],
       [  2,  24,   4,  17, -13,  41,  32,  14,  15,  18,  -7,   0, -19,
         -5],
       [ 25,  35,  -5,  16,  11,  12, -15,  15,  18,  40,  -2,  15, -30,
         31],
       [ 40,  52,  19,  36,   5,   7,  35,   4, -17,  32,  -3,  14,   1,
         17],
       [ 49,  52, -17,   4,   6,  -6,  -2,   9,  39,  42,  -4,  11,   8,
        -14],
       [ 50,  -5,  10,  21,  16, -18,  40,  39,  22,   0,  31,  -9,  15,
         -6],
       [ 14,  31,  20,   9,  30,  31,  42, 

### Prepare scan chains for multicast ID

In [179]:
ids_acts = np.zeros((nOuts+1,nWeights))
for x,idx in enumerate(ids_acts):
    for y,id in enumerate(idx):
        ids_acts[x][y] = x + y
ids_acts[-1] = [0,0,0] # Only zeros for the Y-bus for now (you only need it for multichannel convolutions)
ids_acts

array([[ 0.,  1.,  2.],
       [ 1.,  2.,  3.],
       [ 2.,  3.,  4.],
       [ 3.,  4.,  5.],
       [ 4.,  5.,  6.],
       [ 5.,  6.,  7.],
       [ 6.,  7.,  8.],
       [ 7.,  8.,  9.],
       [ 8.,  9., 10.],
       [ 9., 10., 11.],
       [10., 11., 12.],
       [11., 12., 13.],
       [12., 13., 14.],
       [13., 14., 15.],
       [ 0.,  0.,  0.]])

In [180]:
ids_weights = np.zeros((nOuts+1,nWeights))
for x,idx in enumerate(ids_weights):
    ids_weights[x] = [i for i in range(nWeights)]
ids_weights[-1] = [0,0,0] # Only zeros for the Y-bus for now (you only need it for multichannel convolutions)
ids_weights

array([[0., 1., 2.],
       [0., 1., 2.],
       [0., 1., 2.],
       [0., 1., 2.],
       [0., 1., 2.],
       [0., 1., 2.],
       [0., 1., 2.],
       [0., 1., 2.],
       [0., 1., 2.],
       [0., 1., 2.],
       [0., 1., 2.],
       [0., 1., 2.],
       [0., 1., 2.],
       [0., 1., 2.],
       [0., 0., 0.]])

In [181]:
tag_order_acts = []
for i in range(nActs):
    tag_order_acts.append([0,i])
tag_order_acts = np.array(tag_order_acts)

tag_order_weights = []
for i in range(nWeights):
    tag_order_weights.append([0,i])
tag_order_weights = np.array(tag_order_weights)

### Save stimulus:

In [182]:
path = '/home/lquizon/lawrence-workspace/eyeriss/tb/cluster'
np.savetxt(path + '/a.txt',a,'%i')
np.savetxt(path + '/w.txt',w,'%i')
np.savetxt(path + '/o.txt',o,'%i')
np.savetxt(path + '/ids_acts.txt',ids_acts,'%i')
np.savetxt(path + '/ids_weights.txt',ids_weights,'%i')
np.savetxt(path + '/tag_order_acts.txt',tag_order_acts,'%i')
np.savetxt(path + '/tag_order_weights.txt',tag_order_weights,'%i')

### Have a look at existing stimulus:

In [183]:
import numpy as np
path = '.'
a = np.loadtxt(path + '/a.txt')
w = np.loadtxt(path + '/w.txt')
o= np.loadtxt(path+'/o.txt')  

### Row-stationary Forensics

In [184]:
osize = nActsX-nWeights+1
sequence_array = [[
    (0, 0), # 0,0
    (1, 1), # 
    (2, 2),
    ],[
    (0, 1),
    (1, 2),
    (2, 3),
    ],[
    (0, 2),
    (1, 3),
    (2, 4)
    ]
]
ps = []
psr = []

for sequence in sequence_array:
    psr = []
    for wrow,arow in sequence:
        psr.append(np.convolve(a[arow],w[wrow][::-1],'valid'))   
    ps.append(psr)

np.array(ps)

# Expected PE partial sums:

array([[[  6.,  -1.,  14., -10.,   8.,  -4.,  17.,  -4.,  23.,  14.,
           1.,   9.,  -7.,  -7.],
        [ 10.,   8.,  18.,   4.,  14.,   8.,  16.,  20.,  18.,  10.,
           6.,  18.,  16.,  12.],
        [  9.,  14.,  20.,  20.,   6.,  16.,  -1.,  14.,  -1.,   4.,
          10.,   0.,  15.,  17.]],

       [[ 19.,   6.,  23.,  -4.,  33.,  -8.,  32.,   8.,  25.,  13.,
          15.,  19.,   8.,  22.],
        [ 14.,  22.,   8.,  16.,   0.,  12.,   4.,  -2.,  12.,  -2.,
           4.,  20.,   4.,  10.],
        [ 15.,  13.,  -5.,  -3.,   4.,   0.,   7.,  -4.,  -5.,   2.,
          23.,  21.,  11.,  -3.]],

       [[ 15.,  17.,   8.,  32., -12.,  34., -16.,  17.,   8., -11.,
          22.,   6.,   0.,  29.],
        [  4.,  -6.,   2.,  -2.,   6.,   4.,  -4., -10.,   8.,  22.,
          20.,   2.,   2.,  -2.],
        [  0.,   7.,  17.,   1.,  14.,   7.,  24.,  15.,  10.,   2.,
          11., -10.,  -5.,  -1.]]])

In [185]:
ress = []
for psr in ps:
    res = psr.copy()
    for i in range(1,len(res)):
        res[i] += res[i-1]
    ress.append(res)
np.array(ress)

# Expected partial sum outputs in systolic passing:

array([[[  6.,  -1.,  14., -10.,   8.,  -4.,  17.,  -4.,  23.,  14.,
           1.,   9.,  -7.,  -7.],
        [ 16.,   7.,  32.,  -6.,  22.,   4.,  33.,  16.,  41.,  24.,
           7.,  27.,   9.,   5.],
        [ 25.,  21.,  52.,  14.,  28.,  20.,  32.,  30.,  40.,  28.,
          17.,  27.,  24.,  22.]],

       [[ 19.,   6.,  23.,  -4.,  33.,  -8.,  32.,   8.,  25.,  13.,
          15.,  19.,   8.,  22.],
        [ 33.,  28.,  31.,  12.,  33.,   4.,  36.,   6.,  37.,  11.,
          19.,  39.,  12.,  32.],
        [ 48.,  41.,  26.,   9.,  37.,   4.,  43.,   2.,  32.,  13.,
          42.,  60.,  23.,  29.]],

       [[ 15.,  17.,   8.,  32., -12.,  34., -16.,  17.,   8., -11.,
          22.,   6.,   0.,  29.],
        [ 19.,  11.,  10.,  30.,  -6.,  38., -20.,   7.,  16.,  11.,
          42.,   8.,   2.,  27.],
        [ 19.,  18.,  27.,  31.,   8.,  45.,   4.,  22.,  26.,  13.,
          53.,  -2.,  -3.,  26.]]])